In [6]:
import pandas as pd
import numpy as np

# Chargement des fichiers originaux
df_metal = pd.read_csv("/home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_metal.csv")

# Mise à jour des noms de colonnes pour correspondre à ceux fournis par l'utilisateur
# df_unique_cotes_metal = df_metal.drop_duplicates(subset="Nom_Cote", keep="first").copy()


In [38]:
def simuler_mesures_corrigee_nominal_base(
    df_metal,
    nom_prefixe="Cire_",
    surcote_mm=0.15,
    bruit_mesure_mm=0.01,
    resserrement_tol=0.5
):
    lignes = []

    for _, row in df_metal.iterrows():
        nom_cote = row["Nom_Cote"]
        serial = row["Serial"]
        of = row["OF"]
        valeur_nominale = row["Nominal"]
        tol_plus = row["Tolérance_Max"] - valeur_nominale
        tol_moins = row["Tolérance_Min"] - valeur_nominale

        # Nouvelle nominale avec surcote
        nouvelle_nominale = valeur_nominale + surcote_mm

        # Tolérances resserrées autour de la nouvelle valeur nominale
        nouvelle_tol_plus = round(nouvelle_nominale + (tol_plus * resserrement_tol), 3)
        nouvelle_tol_moins = round(nouvelle_nominale + (tol_moins * resserrement_tol), 3)

        # Sigma basé sur tolérance
        sigma = min(nouvelle_tol_plus - nouvelle_nominale, abs(nouvelle_tol_moins - nouvelle_nominale)) / 3
        if sigma <= 0:
            sigma = 0.01

        # Mesure simulée autour de la nouvelle nominale
        mesuree = round(np.random.normal(loc=nouvelle_nominale, scale=sigma), 3)
        mesuree += round(np.random.normal(loc=0, scale=bruit_mesure_mm), 3)

        lignes.append({
            "Date": pd.Timestamp.today().normalize(),
            "Serial": serial,
            "OF": of,
            "Nom_Cote": nom_prefixe + nom_cote,
            "Mesure": mesuree,
            "Nominal": nouvelle_nominale,
            "Tolérance_Min": nouvelle_tol_moins,
            "Tolérance_Max": nouvelle_tol_plus
        })

    return pd.DataFrame(lignes)




In [39]:

# Fusion avec les données d'origine
df_simule_cire = simuler_mesures_corrigee_nominal_base(df_metal)

output_path_cire = "/home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_cire.csv"
df_simule_cire.to_csv(output_path_cire, index=False)
print(f"✅ Fichier exporté : {output_path_cire}")

✅ Fichier exporté : /home/riccardo/Visual_Studio_Code/freelance/Synthetique_data/simulation_mesures_cire.csv
